In [1]:
%%capture
!pip install pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode datasets transformers

In [6]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM
import torchaudio.functional as F

In [10]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


In [16]:
sample_iter = iter(load_dataset("common_voice", "sv-SE", split="test", streaming=True, use_auth_token=True))

sample = next(sample_iter)
resampled_audio = F.resample(torch.tensor(sample["audio"]["array"]), 48_000, 16_000).numpy()

processor = Wav2Vec2ProcessorWithLM.from_pretrained("birgermoell/lm-swedish")
#processor = Wav2Vec2Processor.from_pretrained("birgermoell/lm-swedish")
model = Wav2Vec2ForCTC.from_pretrained("birgermoell/lm-swedish")

#input_values = processor(resampled_audio, return_tensors="pt").input_values
input_values = processor(resampled_audio, return_tensors="pt")

with torch.no_grad():
#    logits = model(input_values).logits
    logits = model(input_values.input_values, attention_mask=input_values.attention_mask).logits
#predicted_ids = torch.argmax(logits, dim=-1)
#print("Prediction:", processor.batch_decode(predicted_ids))


transcription = processor.batch_decode(logits.numpy()).text

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:354: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [17]:
transcription 

['ÄGX YNUFGÅ9 X3AIÄA</s></s>90 B0PAFAU']

In [ ]:
from datasets import load_metric
wer = load_metric("wer")
wer_result = wer.compute(references=result["target"], predictions=transcription["prediction"])
